# Common Mistake (Writeup)
###### _Categoría: Crypto_
###### _by: Danex_
###### _Dificultad: Fácil_
###### _Enunciado:_  
Elves are trying very hard to communicate in perfect secrecy in order to keep Santa's warehouse. Unfortunately, their lack of knowledge about cryptography leads them to common mistakes.
###### _Descarga los archivos en el repositorio oficial de H3xTEL [aquí](https://github.com/D-Cryp7/H3xTEL)_ :D

A través del archivo `encrypted.txt` nos damos cuenta de que se utilizaron 2 llaves públicas $(n, e_1)$ y $(n, e_2)$ para cifrar mensajes. Un error común (_common mistake_) es que el mismo mensaje $m$ se cifre utilizando el mismo módulo $n$ pero con exponentes públicos $e$ distintos. Este comportamiento no es anormal, ya que una falla en la transmisión de un paquete puede causar que se reenvíe, pero con la mala práctica de que se reutilice el módulo $n$, esto debido a que podemos recuperar el mensaje original $m$ de la siguiente forma:
* Primer mensaje cifrado:
\begin{equation}
    c_1 \equiv m ^ {e_1} \text{ mod } n
\end{equation}
* Segundo mensaje cifrado:
\begin{equation}
    c_2 \equiv m ^ {e_2} \text{ mod } n
\end{equation}
* El _common modulus attack_ consiste en multiplicar ambos mensajes cifrados, pero de una forma ingeniosa. Para ello, se debe cumplir que
\begin{equation}
    \text{gcd}(e_1, e_2) = 1 \implies s \cdot e_1 + t \cdot e_2 = 1
\end{equation}
es decir, que el máximo común divisor (_great common divisor_) sea igual a 1, o que es lo mismo, que $e_1$ y $e_2$ sea coprimos. Así, multiplicamos los 2 mensajes cifrados de la siguiente manera:
\begin{equation}
    c_{1}^{s} \cdot c_{2}^{t} \equiv m ^ {s \cdot e_1 + t \cdot e_2} \text{ mod } n
\end{equation}
y por la propiedad anterior
\begin{equation}
    c_{1}^{s} \cdot c_{2}^{t} \equiv m ^ {1} = m \text{ mod } n
\end{equation}
* Así pues, vamos a asumir que esto se puede realizar, y lo vamos a probar para obtener la _flag_

In [4]:
# leer encrypted.txt

import ast
from Crypto.Util.number import bytes_to_long, long_to_bytes
import math
f = open('encrypted.txt', 'r').read().split("\n")
enc1 = ast.literal_eval(f[0])
enc2 = ast.literal_eval(f[1])
for key, value in enc1.items():
    if key == 'e':
        enc1[key] = int(enc1[key][2:], 16)
        enc2[key] = int(enc2[key][2:], 16)
    else:
        enc1[key] = bytes_to_long(bytes.fromhex(enc1[key][2:]))
        enc2[key] = bytes_to_long(bytes.fromhex(enc2[key][2:]))

In [5]:
# comprobar que s y t son coprimos
# los valores de s y t se obtienen a través del Extended Euclidean Algorithm

def eea(r0,r1):
    if r0 == 0:
        return (r1, 0, 1)
    else:
        g, s, t = eea(r1 % r0, r0)
        return (g, t - (r1 // r0) * s, s)
g, s1, s2 = eea(enc1['e'], enc2['e'])
enc1['e']*s1 + enc2['e']*s2

1

In [7]:
# Se obtiene la flag

m = (pow(enc1['ct'], s1, enc1['n']) * pow(enc2['ct'], s2, enc2['n'])) % enc1['n']
flag = long_to_bytes(m)
flag

b'HTB{c0mm0n_m0d_4774ck_15_4n07h3r_cl4ss1c}'

ref: https://crypto.stackexchange.com/questions/16283/how-to-use-common-modulus-attack